In [ ]:
!pip install -r requirements.txt -q

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
from transformers import pipeline
import nltk
import collections
from IPython.display import Image, display
from tqdm import tqdm

/home/heleon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-07 07:20:26.042671: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 07:20:29.419811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Loading dataset
# Must create a kaggle account and auth token here: https://www.kaggle.com/settings/account
od.download("https://www.kaggle.com/datasets/jonathanbesomi/superheroes-nlp-dataset")
df = pd.read_csv('superheroes-nlp-dataset/superheroes_nlp_dataset.csv')

Skipping, found downloaded files in "./superheroes-nlp-dataset" (use force=True to force download)


## Question 1: 
### How would you define the most powerful superhero from the information available in dataset?

In [65]:
# Sort rows by overall score of Superhero
most_powerful_superhero = df.sort_values(by="overall_score", ascending=False)["name"][0]
print(f"Most Powerful Superhero (overall score): {most_powerful_superhero}")

# Get their score and find others who share it
best_score = df.sort_values(by="overall_score", ascending=False)["overall_score"][0]
tied_first = df[df["overall_score"]==best_score]["name"].tolist()
tied_first.remove(most_powerful_superhero)
if tied_first != []:
    print(f"  - Tied for first: {', '.join(tied_first)}")

Most Powerful Superhero (overall score): 3-D Man
  - Tied for first: Agent Zero (FOX), Ajax, Alien, Amanda Waller, Anacondrai Serpent, Annabeth Chase (FOX), Archangel, Ariel, Armor, Artemis (CW), Batgirl IV, Batgirl VI, Batman (1966), Beast Boy, Big Man, Bill Harken, Black Canary, Black Cat, Black Knight III, Black Lightning (CW), Blade, Blob (FOX), Bloodhawk, Brundlefly, Buffy, Callisto, Captain Boomerang (CW), Captain Boomerang (DCEU), Captain Metropolis, Captain Soto, Catwoman, Changeling, Chuck Norris, Claire Bennet, Clock King, Colossus (FOX), Crystal, Dagger, Darkhawk, Deadshot (CW), Deadshot (New 52), Destroyer (MCU), Doubloon, Dove (Titans), Echo Zane, Elektra (MCU), Elektra, Ethan Hunt, Evilhawk, Fandral (MCU), Fangtom, Firelord, Firestar, Gambit, Gary Bell, Green Arrow (Injustice), Gypsy (CW), Happy Hogan, Havok, Hawkwoman III, Hellcat, Huntress (Injustice), Iceman (FOX), Indiana Jones, James T. Kirk, Jesse Custer, Jessica Jones, Jigsaw (MCU), Joe Wilson (CW), John Pilgrim (M

### Who is the most powerful superhero of each creator?

In [70]:
print("Most Powerful Superheros (overall score):")
creators = df["creator"].unique()
creators = [str(creator)for creator in creators  if str(creator) != "nan" ]

# Loop through creators and sort rows with that creator by overall score of superhero
for creator in creators:
    creator_df = df[df["creator"] == creator]
    most_powerful_superhero = creator_df.sort_values(by="overall_score", ascending=False)["name"].tolist()[0]
    print(f"- {creator}: {most_powerful_superhero}")

    # Repeat the code for finding ties
    best_score = creator_df.sort_values(by="overall_score", ascending=False)["overall_score"].tolist()[0]
    tied_first = creator_df[creator_df["overall_score"]==best_score]["name"].tolist()
    tied_first.remove(most_powerful_superhero)
    if tied_first != []:
        print(f" - Tied for first: {', '.join(tied_first)}")

Most Powerful Superheros (overall score):
- Marvel Comics: Living Tribunal
 - Tied for first: Abraxas, Beyonder, The Beyonder (Earth-1298), The One Below All, True Form Oblivion
- DC Comics: Death Of The Endless
 - Tied for first: Black Alice, Life Entity, Michael Demiurgos, Spectre Oversoul
- George Lucas: Darth Nihilus
- Dark Horse Comics: Johann Krauss
 - Tied for first: Angel, T-850
- Blizzard Entertainment: A'dal
- NBC - Heroes: Adam Monroe
- Lego: Ronin (Ninjago)
 - Tied for first: GPL, Nya (The LEGO Ninjago Movie)
- Wildstorm: Allan Quatermain
 - Tied for first: Sam Winchester
- Ubisoft: Altaïr Ibn-La'Ahad
 - Tied for first: Desmond Miles, Edward Kenway, Ezio Auditore da Firenze
- J. R. R. Tolkien: Eru Iluvatar
- Image Comics: Man Of Miracles
- George R. R. Martin: The Lord Of Light
- Capcom: Asura
- Icon Comics: Hit-Girl
- SyFy: Bill Harken
 - Tied for first: Gary Bell, Nina Theroux
- Hanna-Barbera: Birdman
- Hasbro: Bumblebee
- Clive Barker: Pinhead
- Team Epic TV: Captain Epi

## Question 2: 
### Find the top 5 superpowers in descending order.

In [64]:
# Define superpowers as the "has" characteristics
superpowers = df[[col for col in df.columns if "has_" in col]]

# Sort characteristics by sum of their occurences
top_5_superpowers = superpowers.sum().sort_values(ascending=False).head(5).keys()
print("Top 5 Most Common Superpowers:\n- "+'\n- '.join(top_5_superpowers))

Top 5 Most Common Superpowers:
- has_agility
- has_stamina
- has_super_strength
- has_durability
- has_intelligence


## Question 3: 
### Which race has the most immortal superheroes?

In [11]:
# Find only rows with immortality and sort races by their value counts 
most_immortal_race = df[df['has_immortality']==True]['type_race'].value_counts().idxmax()
print(f"Most Immortal Race: {most_immortal_race}")

Most Immortal Race: God / Eternal


## Question 4: 
### Name the creator having most superheroes of type “Parademon”.

In [12]:
# Find only rows with Parademons and sort creators by their value counts 
creator_max_parademon = df[df['type_race']=='Parademon']['creator'].value_counts().idxmax()
print(f"Creator with most Parademons: {creator_max_parademon}")

Creator with most Parademons: DC Comics


## Question 5: 
### Which comic creator has the most superhero teams?

In [13]:
creators = df["creator"].unique()
creators = [str(creator)for creator in creators  if str(creator) != "nan" ]
num_teams = []
creator_teams = []

# Loop through creators
for creator in creators:
    all_teams = []

    # Add teams from each row to all teams list
    for teams in df[df["creator"]==creator]["teams"]:
        all_teams.extend(eval(teams))

    # Find all unique teams
    all_teams = list(set(all_teams))
    num_teams.append(len(all_teams))
    creator_teams.append(all_teams)

# Find creator with most unique teams
creator_most_teams = creators[np.argmax(num_teams)]
max_teams = max(num_teams)
print(f"Creator with the most teams: {creator_most_teams} ({max_teams} teams)")

Creator with the most teams: Marvel Comics (165 teams)


### Find names, real names and alias of superhero who is part of most teams.

In [14]:
# Create a column that is equal to the length of the team list of that row
df["num_teams"] = [len(eval(team_list)) for team_list in df['teams'].tolist()]

# Sort and find the name, real name, and aliases of the superhero on the most teams
name = df.sort_values(by="num_teams", ascending=False)["name"][0]
real_name = df.sort_values(by="num_teams", ascending=False)["real_name"][0]
aliases = eval(df.sort_values(by="num_teams", ascending=False)["aliases"][0])
print(f"Superhero on the most teams:\n- Name: {name}\n- Real name: {real_name}\n- Aliases:{' '.join(aliases)}")

Superhero on the most teams:
- Name: 3-D Man
- Real name: Delroy Garrett, Jr.
- Aliases:


### Are there any crossovers between creators and teams?

In [15]:
found_pairs = []

# Loop through all combinations of creator's teams and check to see
# if they share any members
for i in range(len(creator_teams)):
    for j in range(len(creator_teams)):
        num_overlaps = len(list(set(creator_teams[i]).intersection(set(creator_teams[j]))))

        # Print out shared members unless the pair has already been printed
        if num_overlaps != 0 and i!=j and not any([pair in found_pairs for pair in [(j,i),(i,j)]]):
            print(f"{creators[i]} shares team members with {creators[j]}")
            found_pairs.append((i,j))

Marvel Comics shares team members with DC Comics
Dark Horse Comics shares team members with Disney


## Question 6
### What are the characteristics that can predict a superhero alignment?

In [71]:
alignments = df["alignment"].unique()
alignments = [str(alignment)for alignment in alignments  if str(alignment) != "nan" ]

characteristics = [column for column in df.columns if "has" in column]
alignment_dict = {}

# Loop through all alignments
for alignment in alignments:
    aligned_df = df[df["alignment"]==alignment]
    alignment_dict[alignment] = {}

    # Loop through characteristics
    for characteristic in characteristics:
        # Find the proportion of characters with that characteristics who 
        # have the given alignment
        total = df[characteristic].sum()
        num_true = aligned_df[characteristic].sum()
        proportion_true = num_true/total * 100
        alignment_dict[alignment][characteristic] = proportion_true

n = 3 

# For each alignment, print the characteristic that has the highest proportion
for alignment in alignment_dict:
    sorted_characteristics = sorted(alignment_dict[alignment].items(), key=lambda kv: kv[1], reverse=True)
    top_characteristics = sorted_characteristics[:n]
    for characteristic, proportion in top_characteristics:
        print(f"If a superhero {characteristic.replace('_',' ')}, they are {round(proportion,2)}% likely to be {str(alignment).lower()}.")
    print()

If a superhero has flight, they are 57.45% likely to be good.
If a superhero has telepathy, they are 56.38% likely to be good.
If a superhero has stealth, they are 56.21% likely to be good.

If a superhero has reality warping, they are 46.15% likely to be bad.
If a superhero has mind control, they are 45.61% likely to be bad.
If a superhero has telekinesis, they are 41.55% likely to be bad.

If a superhero has reality warping, they are 18.46% likely to be neutral.
If a superhero has self-sustenance, they are 17.65% likely to be neutral.
If a superhero has illusions, they are 17.6% likely to be neutral.



## Question 7
### From history of superheroes,
a). Find list of superheroes having negative past but now aligned positively.
(Negativity of past can be decided by multiple methods, please use which is
familiar to you)

In [17]:
# Find all heroes with valid histories and good alignment
good_df = df[df["history_text"].notna()][~df["history_text"].isin(["History unknown.","Unknown"])][df.alignment=="Good"]

# Shortening df because I'm not connected to a GPU
good_df = good_df[:200]

# Set up sentiment analysis pipeline from huggingface
pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}

/tmp/ipykernel_32430/3511450926.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  good_df = df[df["history_text"].notna()][~df["history_text"].isin(["History unknown.","Unknown"])][df.alignment=="Good"]
/tmp/ipykernel_32430/3511450926.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  good_df = df[df["history_text"].notna()][~df["history_text"].isin(["History unknown.","Unknown"])][df.alignment=="Good"]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [18]:
# Run inference on the histories
histories = good_df["history_text"].tolist()
results = pipe(histories, **tokenizer_kwargs)
sentiment_labels = [r["label"] for r in results]
good_df["history_sentiment"] = sentiment_labels

In [19]:
# Find heroes who had negative pasts
neg_labels = ["1 star", "2 stars"]
overcame_adversity = good_df[good_df.history_sentiment.isin(neg_labels)]["name"].tolist()
print(f"These heroes overcame their negative pasts: {', '.join(overcame_adversity)}")

These heroes overcame their negative pasts: Chuck Norris


b). Extract patterns from superhero history for each creator.

In [20]:
# Define function for finding ngram counts of a requested length using nltk
def find_patterns(histories, num_results, ngrams=5):
    all_histories = ' '.join(histories)
    words = nltk.word_tokenize(all_histories)
    patterns = nltk.ngrams(words, ngrams)
    pattern_frequency = collections.Counter(patterns)
    return pattern_frequency.most_common(n)

creators = df["creator"].unique()
creators = [str(creator)for creator in creators  if str(creator) != "nan" ]
pattern_dict = {}

# Loop through creators and find the top 3 ngrams of the default length 5
for creator in creators:
    creator_df = df[df["history_text"].notna() & ~(df["history_text"].isin(["History unknown.","Unknown"])) & (df.creator==creator)]
    histories = creator_df["history_text"]
    patterns = find_patterns(histories, 3)
    pattern_dict[creator] = patterns

# Here is an example of the patterns for Marvel
pattern_dict["Marvel Comics"]

[(('.', 'During', 'this', 'time', ','), 42),
 (('.', 'As', 'a', 'result', ','), 30),
 (('and', 'the', 'rest', 'of', 'the'), 26)]

## Question 8
### Report on the 10 superheroes with most relatives, status of those relatives where possible, and the alignment of those superheroes.

In [21]:
# Create python lists out of the strings of relatives
relatives = [str(string).split(", ") for string in df["relatives"].tolist()]
df["relatives"] = relatives

# Create a column for the lengths fo the relative lists to determine
# number of relatives
df["num_relatives"] = [len(r) for r in relatives]

# Sort by num_relatives and find the names of superheros with the most
# relatives
most_relatives = df.sort_values(by="num_relatives", ascending=False)["name"][:10]

In [74]:
# Loop through the heroes with the top 10 most relatives
for hero in most_relatives:
    # Find the names of all their relatives by cleaning the strings
    relatives = df[df["name"]==hero]["relatives"].tolist()[0]
    names = [relative[:relative.find("(")].strip() for relative in relatives]

    # Print a list of all of the names
    print(f"{hero}'s relatives: {', '.join(names)}")

    # Loop through all names (with and without the relation included)
    for relative, name in zip(relatives, names):

        # Display information found about names if available
        if name in df["name"].tolist():
            report = f"- {relative}"
            status = str(df[df["name"]==name]["powers_text"].tolist()[0])
            status = status[:status.find(".")+1]
            report += f"\n  - Status: {status}"
            alignment = str(df[df["name"]==name]["alignment"].tolist()[0])
            report += f"\n  - Alignment: {alignment}"
            print(report)
    print()            

Namor's relatives: Elanna, Tanas, Zartra, Orrek, Stegor, Kamuu, Harran, Kalen, Ossem, Balaal, Thallo, Immanu, deceased, Thakorr, deceased, Korra, Wa-Korr, presumed deceased, Wathan, presumed deceased, Tom Smallwood, Gladys Smallwood, Zarina, Namora's father, Daka, Brynn, Fen, deceased, Leonard McKenzie, deceased, Experiment N2, Lawrence McKenzie, Dorma, deceased, Marrina, three unnamed children, Kamar, deceased, Black Moray, Llyron McKenzie, Dan Smallwood, Namora, Beemer, Bobo, Byrrahna, Namita, Daro, Dara, Seth, Argus, Arkus, Byrrah, Namorita
- Namora (maternal cousin)
  - Status: Namora is biologically half Atlantean and half human with a mutant X-Gene.
  - Alignment: Good
- Namorita (maternal 1st cousin once removed).
  - Status: Namorita possesses superhuman strength, speed, durability, agility, and reflexes; she can lift up to 75 tons underwater, although prolonged surface activity can reduce her strength to 33% or lower.
  - Alignment: Good

Havok's relatives: Oscar Summers, dece

## Question 9
### Find out any other interesting insights from the given data.

In [24]:
# Create a list of all unique teams
teams = [eval(list) for list in df["teams"].tolist()]
all_teams = []
for team_list in teams:
    all_teams.extend(team_list)
all_teams = list(set(all_teams))

team_to_members = {}
members_to_teams = {}

# Create a dictionary linking the team members to all their teams
for team in all_teams:
    member_df = df[df["teams"].str.contains(team)]
    members = member_df["name"].tolist()
    team_to_members[team] = members
    for member in members:
        try:
            members_to_teams[member].append(team)
        except:
            members_to_teams[member] = [team]


In [26]:
# Define a function for finding nth degree connections between team members
# where a 1st degree direct connection is if they've worked on a team 
# together, a 2nd degree is if they've worked with the same person, and so on
def connections_loop(member, n):
    visited = [] 
    queue = [(member, 0)]

    while queue:
        current_member, degree = queue.pop(0)

        if degree > n:
            break

        if current_member not in visited:
            visited.append(current_member)

            for team in members_to_teams[current_member]:
                for next_member in team_to_members[team]:
                    queue.append((next_member, degree + 1))

    n_degree_connections = [m for m in visited if m != member]

    return n_degree_connections

# Define a wrapper for the previous function so that it only includes nth degree connections
# and no connections of a degree less than n
def find_connections(member, n):
    return list(set(connections_loop(member, n))-set(connections_loop(member, n-1)))

# Find the farthest connections for each hero
biggest_links = {}
for member in tqdm(members_to_teams.keys()):
    farthest_link = 1
    results = find_connections(member, 1)
    while results != []:
        results = find_connections(member, farthest_link)
        if results != []:
            farthest_link += 1
        else:
            biggest_links[member] = farthest_link
    

  0%|          | 0/550 [00:00<?, ?it/s]

100%|██████████| 550/550 [12:11<00:00,  1.33s/it]


In [44]:
members = []
link_lengths = []

for member, link_length in biggest_links.items():
    members.append(member)
    link_lengths.append(link_length)

# Find the heroes with the farthest connection and share info
member_farthest_connection = members[np.argmax(link_lengths)]
farthest_connection = link_lengths[np.argmax(link_lengths)]

connection = find_connections(member_farthest_connection, farthest_connection-1)[0]

# This connection is interesting, because Percy Jackson isn't usually considered a superhero,
# yet he is technically part of the DC Universe
print(f"{member_farthest_connection} from {df[df['name']==member_farthest_connection]['creator'].tolist()[0]} has a {farthest_connection-1}th degree connection with {connection} from {df[df['name']==connection]['creator'].tolist()[0]}")


Rick Flag from DC Comics has a 7th degree connection with Percy Jackson from nan


### Which 3 comic characters can you recommend to your friends to read or watch?

In [ ]:
!pip install perspectives==1.0.8 -q

In [ ]:
import perspectives as ps

# Perspecives is a python library that I made which allows semantic search over pandas dataframes
# It is much faster on GPUs
search_df = df[["name", "occupation"]]
search_df["occupation"] = [str(o) for o in search_df["occupation"].tolist()]
perspectives_df = ps.DataFrame(search_df)

In [101]:
# My friends really like music, so I did a semantic for heroes who are musicians
perspectives_df.search(occupation="Musician").head()

/home/heleon/.local/lib/python3.10/site-packages/perspectives/DataFrame.py:111: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  setattr(self, key + "_embs", self._embmodel.encode(self[key].tolist()))


,name,occupation,text
0,Black Cat (Earth 65),"Musician, thief",None
1,The Crow,Heavy Metal Guitarist,None
2,Red Arrow,Adventurer; intelligence agent; musician,None
3,Herald,"Adventurer, Composer",None
4,Pied Piper (CW),Physicist,None


In [105]:
# My brother is a teacher, so I did a semantic for heroes who are teachers
perspectives_df.search(occupation="teacher").head()

,name,occupation,text
0,Bookworm,Teacher,None
1,Wolfsbane,Teacher,None
2,Tom Corsi,Teacher; formerly police officer,None
3,Black Lightning (CW),"Teacher, Vigilante",None
4,Static,Student,None


In [108]:
# My little brother wants to be an astronaut, so I did a semantic for heroes who work in space
perspectives_df.search(occupation="astronaut").head()

,name,occupation,text
0,Morty Smith,"Student, Space Adventurer",None
1,Doomguy,US Space Marine Captain,None
2,Doom Slayer,US Space Marine Captain,None
3,Cosmo The Spacedog,"Astronaut, liason for the new Guardians of the...",None
4,Darkman,Scientist,None
